# MFT Asynchronous QC

In [1]:
import ROOT
from ROOT import gStyle

_period = "LHC22o"
_run = 527863
_pass1 = "apass1"
_pass2 = "apass2"
print(f"period: {_period}")
print(f"run:    {_run}")
print(f"pass1:  {_pass1}")
print(f"pass2:  {_pass2}")

Welcome to JupyROOT 6.26/10
period: LHC22o
run:    527863
pass1:  apass1
pass2:  apass2


## TRACK POSITION:

In [2]:
names = [
# track position (6 in total)
"tracks_mMFTTrackEtaPhi_5_MinClusters",
"tracks_mMFTTrackXY_5_MinClusters",
"tracks_mMFTTrackEtaPhi_7_MinClusters",
"tracks_mMFTTrackXY_7_MinClusters",
"tracks_mMFTTrackEtaPhi_8_MinClusters",
"tracks_mMFTTrackXY_8_MinClusters",
# tracks (11 in total)
"tracks_mMFTTrackEta",
"tracks_mMFTTrackNumberOfClusters",
"tracks_mMFTTrackPhi",
"tracks_mMFTTrackTanl",
"tracks_mNOfTracksTime",
"tracks_mMFTTrackInvQPt",
"tracks_CA_mMFTCATrackPt",
"tracks_LTF_mMFTLTFTrackPt",
"tracks_CA_mMFTCATrackEta",
"tracks_LTF_mMFTLTFTrackEta",
"tracks_mMFTTracksBC",
# clusters (4 in total)
"clusters_mMFTClusterPatternIndex",
"clusters_mMFTClusterSensorIndex",
"clusters_mMFTClustersROFSize",
"clusters_mNOfClustersTime"]

f_p1 = ROOT.TFile.Open(f"Results/runsRootFiles/{_period}/{_run}_{_pass1}.root")
f_p2 = ROOT.TFile.Open(f"Results/runsRootFiles/{_period}/{_run}_{_pass2}.root")
%jsroot on

gStyle.SetTitleX(0.4)
gStyle.SetTitleY(1.0)
gStyle.SetOptStat(0)

c1_cols = 3
c1_rows = 2
c1_figs = c1_cols*c1_rows
c1 = ROOT.TCanvas("c1","",c1_cols*400,c1_rows*300)
c1.Divide(c1_cols,c1_rows)
c1.Draw()

## TRACK PROPERTIES:

In [9]:
c2a_cols = 3
c2a_rows = 2
c2a_figs = c2a_cols*c2a_rows
c2a = ROOT.TCanvas("c2a","",c2a_cols*400,c2a_rows*300)
c2a.Divide(c2a_cols,c2a_rows)
for i in range(6,6+c2a_figs):
    h_p1 = f_p1.Get(names[i])
    h_p2 = f_p2.Get(names[i])
    c2a.cd((i-6)+1)
    if h_p1 != None: 
        h_p1.SetLineColor(4)
        h_p1.Scale(1./h_p1.Integral())
        h_p1.Draw("HIST")
    if h_p2 != None: 
        h_p2.SetLineColor(2)
        h_p2.Scale(1./h_p2.Integral())
        h_p2.Draw("HIST SAME")
c2a.Draw()

c2b_cols = 3
c2b_rows = 2
c2b_figs = c2b_cols*c2b_rows
c2b = ROOT.TCanvas("c2b","",c2b_cols*400,c2b_rows*300)
c2b.Divide(c2b_cols,c2b_rows)
for i in range(12,17):
    h_p1 = f_p1.Get(names[i])
    h_p2 = f_p2.Get(names[i])
    c2b.cd((i-12)+1)
    if h_p1 != None: 
        h_p1.Draw()
    if h_p2 != None: 
        h_p2.Draw("SAME")
c2b.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2a
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2b


## CLUSTER PROPERTIES:

In [4]:
c3_cols = 2
c3_rows = 2
c3 = ROOT.TCanvas("c3","",c3_cols*400,c3_rows*300)
c3.Divide(2,2)
for i in range(17,21):
    h_p1 = f_p1.Get(names[i])
    h_p2 = f_p2.Get(names[i])
    c3.cd((i-17)+1)
    if h_p1 != None: 
        h_p1.Draw()
    if h_p2 != None: 
        h_p2.Draw("SAME")
c3.Draw()